In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
# --- TAMBAHAN IMPORT MODEL ---
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# --- --------------------- ---
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, f1_score
import warnings
import time

warnings.filterwarnings('ignore')

# ---------------------------------------------------------------------------
# LANGKAH 1: SETUP LINGKUNGAN
# ---------------------------------------------------------------------------
print("Langkah 1: Menghubungkan Google Drive...")
try:
    drive.mount('/content/drive')
    print("Google Drive berhasil terhubung.")
except Exception as e:
    print(f"Error menghubungkan Drive: {e}")
    # Berhenti jika drive tidak terhubung
    raise

# ---------------------------------------------------------------------------
# LANGKAH 2: PEMUATAN DATA (OPTIMASI RAM + MENGAMBIL 'createdAt')
# ---------------------------------------------------------------------------
print("\nLangkah 2: Optimasi Pemuatan Data...")
file_path = '/content/drive/MyDrive/Finpro DS/All_Subjects_data_compained3.csv'
NROWS_SAMPLE = 5_000_000

print(f"Langkah 2a: Membaca header dari {file_path}...")
try:
    df_header = pd.read_csv(file_path, nrows=1)
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di {file_path}")
    raise

timestamp_cols = [col for col in df_header.columns if 'Timestamp' in col]
COLS_TO_IGNORE = [
    'emotionalMoodLevel', 'Unnamed: 0', 'onWristStatus',
    'watchStatus', 'rukun',
] + timestamp_cols

COLS_WE_NEED = [col for col in df_header.columns if col not in COLS_TO_IGNORE]
print(f"Kolom yang akan dimuat (dioptimasi): {len(COLS_WE_NEED)}")

print(f"Langkah 2c: Memuat {NROWS_SAMPLE} baris (HANYA kolom yang dibutuhkan)...")
try:
    df = pd.read_csv(file_path, nrows=NROWS_SAMPLE, usecols=COLS_WE_NEED)
    print(f"Data berhasil dimuat. Ukuran: {df.shape}")
except Exception as e:
    print(f"Error saat memuat dengan 'usecols': {e}")
    raise

# ---------------------------------------------------------------------------
# LANGKAH 3: ANALISIS DATA KUNCI
# ---------------------------------------------------------------------------
print("\nLangkah 3: Analisis Data Kunci...")
print(f"Jumlah ID: {df['id'].nunique()}")

# ---------------------------------------------------------------------------
# LANGKAH 3.5: (PERBAIKAN) AGREGRASI & PEMBERSIHAN
# ---------------------------------------------------------------------------
print("\nLangkah 3.5: Memulai Pembersihan & Agregasi Data...")

# 3.5a: Konversi Waktu
print("  - Mengonversi 'createdAt' ke datetime...")
df['createdAt'] = pd.to_datetime(df['createdAt'])

# 3.5b: Konversi Paksa Fitur Sensor ke Numerik
print("  - Memaksa kolom sensor menjadi numerik...")
features_to_check = ['heartRate', 'skinTemperature']
for col in features_to_check:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3.5c: (FIX 'KeyError') Agregasi Cerdas
print("  - Mengagregasi data (Metode Cerdas)...")

# Dapatkan SEMUA kolom yang BUKAN 'id' or 'createdAt' (kolom grup kita)
cols_to_agg = [col for col in df.columns if col not in ['id', 'createdAt']]

# Mulai dengan 'mean' untuk SEMUA kolom
agg_dict = {col: 'mean' for col in cols_to_agg}

# SEKARANG, ganti (overwrite) 'physicalTiredLevel' ke 'first'
# Ini adalah KUNCI agar target kita tidak rusak
if 'physicalTiredLevel' in agg_dict:
    agg_dict['physicalTiredLevel'] = 'first'
else:
    # Ini akan memicu error jika tidak dimuat, tapi lebih jelas
    print("FATAL ERROR: 'physicalTiredLevel' tidak ditemukan di df.columns saat membuat agg_dict.")
    raise KeyError("'physicalTiredLevel' not in df.columns")

try:
    df = df.groupby(['id', 'createdAt']).agg(agg_dict).reset_index()
    print(f"  - Ukuran data setelah agregasi: {df.shape}")
except Exception as e:
    print(f"Error saat agregasi cerdas: {e}")
    raise

# 3.5d: SORTING
print("  - Menyortir data berdasarkan id dan waktu...")
df = df.sort_values(by=['id', 'createdAt'])

print("Pembersihan & Agregasi Selesai.")


# ---------------------------------------------------------------------------
# LANGKAH 4: PREPROCESSING & PEMBUATAN TARGET
# ---------------------------------------------------------------------------
print("\nLangkah 4: Preprocessing & Pembuatan Target...")
df['y_binary'] = np.where(df['physicalTiredLevel'] > 1, 1, 0)
y = df['y_binary']
COLS_TO_DROP_FROM_X = ['physicalTiredLevel', 'id', 'y_binary', 'createdAt']
cols_to_drop_existing = [col for col in COLS_TO_DROP_FROM_X if col in df.columns]
X = df.drop(columns=cols_to_drop_existing)
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features)],
    remainder='passthrough'
)
print("Pipeline preprocessing (Imputer + Scaler) siap.")

# ---------------------------------------------------------------------------
# LANGKAH 5: GROUP-BASED SPLIT
# ---------------------------------------------------------------------------
print("\nLangkah 5: Melakukan Group-Based Split (Split Cerdas)...")
distribusi_per_id = df.groupby('id')['y_binary'].mean()
good_test_ids_candidates = distribusi_per_id[(distribusi_per_id > 0.1) & (distribusi_per_id < 0.9)]
if good_test_ids_candidates.empty:
    good_test_ids_candidates = distribusi_per_id[distribusi_per_id > 0.01]
if good_test_ids_candidates.empty:
    raise ValueError("Tidak ada peserta yang valid untuk test set (data 'Lelah' tidak cukup).")

TEST_ID_CHOICE = good_test_ids_candidates.index[0]
TEST_IDS = [TEST_ID_CHOICE]
unique_ids = sorted(df['id'].unique())
TRAIN_IDS = [id for id in unique_ids if id not in TEST_IDS]

train_indices = df['id'].isin(TRAIN_IDS)
test_indices = df['id'].isin(TEST_IDS)
X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]
print(f"Data split selesai. Siap melatih {len(TRAIN_IDS)} peserta, menguji pada ID {TEST_IDS[0]}.")


# ---------------------------------------------------------------------------
# LANGKAH 6: DEFINISI 3 MODEL & PIPELINE
# ---------------------------------------------------------------------------
print("\nLangkah 6: Mendefinisikan 3 Model...")

# Definisikan 3 model. Perhatikan 'class_weight='balanced'' di semua model.
model_lgbm = lgb.LGBMClassifier(
    random_state=42,
    class_weight='balanced'
)

model_logreg = LogisticRegression(
    random_state=42,
    class_weight='balanced',
    max_iter=1000 # Tambahkan max_iter agar konvergen
)

model_rf = RandomForestClassifier(
    n_estimators=100, # 100 pohon (default)
    random_state=42,
    class_weight='balanced',
    n_jobs=-1 # Gunakan semua core CPU
)

# Buat 3 pipeline terpisah, tapi gunakan preprocessor YANG SAMA
pipeline_lgbm = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_lgbm)])
pipeline_logreg = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_logreg)])
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_rf)])

print("3 pipeline model (LGBM, LogReg, RF) siap.")

# ---------------------------------------------------------------------------
# LANGKAH 7: MELATIH & MENGEVALUASI SEMUA MODEL
# ---------------------------------------------------------------------------
print("\nLangkah 7: Memulai Kontes Model...")

# Kamus untuk menyimpan hasil
hasil_f1 = {}
waktu_latih = {}

# --- Model 1: LightGBM ---
print("\n--- Melatih Model 1: LightGBM ---")
start_time = time.time()
pipeline_lgbm.fit(X_train, y_train)
waktu_latih['LGBM'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['LGBM']:.2f} detik.")
y_pred_lgbm = pipeline_lgbm.predict(X_test)
hasil_f1['LGBM'] = f1_score(y_test, y_pred_lgbm, pos_label=1)
print(f"F1-Score LGBM (Lelah): {hasil_f1['LGBM']:.4f}")

# --- Model 2: Logistic Regression ---
print("\n--- Melatih Model 2: Logistic Regression ---")
start_time = time.time()
pipeline_logreg.fit(X_train, y_train)
waktu_latih['LogReg'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['LogReg']:.2f} detik.")
y_pred_logreg = pipeline_logreg.predict(X_test)
hasil_f1['LogReg'] = f1_score(y_test, y_pred_logreg, pos_label=1)
print(f"F1-Score LogReg (Lelah): {hasil_f1['LogReg']:.4f}")

# --- Model 3: Random Forest ---
print("\n--- Melatih Model 3: Random Forest ---")
print("PERINGATAN: Ini akan memakan waktu lama (bisa 10-20+ menit)...")
start_time = time.time()
pipeline_rf.fit(X_train, y_train)
waktu_latih['RF'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['RF']:.2f} detik.")
y_pred_rf = pipeline_rf.predict(X_test)
hasil_f1['RF'] = f1_score(y_test, y_pred_rf, pos_label=1)
print(f"FB1-Score RF (Lelah): {hasil_f1['RF']:.4f}")

# ---------------------------------------------------------------------------
# LANGKAH 8: PERBANDINGAN FINAL & KEPUTUSAN
# ---------------------------------------------------------------------------
print("\n" + "="*50)
print(f" HASIL AKHIR KONTES MODEL ".center(50, "="))
print("="*50)
print("Skor F1 (Kelas 1 'Lelah') - Semakin tinggi semakin baik:")
# Urutkan hasil F1 dari tertinggi ke terendah
hasil_f1_sorted = sorted(hasil_f1.items(), key=lambda item: item[1], reverse=True)
for model, score in hasil_f1_sorted:
    print(f"  - {model:<20}: {score:.4f}")

print("\nWaktu Latih - Semakin rendah semakin baik:")
waktu_latih_sorted = sorted(waktu_latih.items(), key=lambda item: item[1])
for model, wkt in waktu_latih_sorted:
    print(f"  - {model:<20}: {wkt:.2f} detik")
print("="*50)

# Tentukan pemenang
pemenang = hasil_f1_sorted[0][0]
print(f"\n🏆 Model Pemenang (F1-Score Tertinggi): **{pemenang}**")
print("="*50)

# Tampilkan laporan rinci HANYA untuk model pemenang
print(f"\n--- Laporan Klasifikasi Rinci untuk {pemenang} ---")
if pemenang == 'LGBM':
    print(classification_report(y_test, y_pred_lgbm, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))
elif pemenang == 'LogReg':
    print(classification_report(y_test, y_pred_logreg, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))
elif pemenang == 'RF':
    print(classification_report(y_test, y_pred_rf, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))

Langkah 1: Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive berhasil terhubung.

Langkah 2: Optimasi Pemuatan Data...
Langkah 2a: Membaca header dari /content/drive/MyDrive/Finpro DS/All_Subjects_data_compained3.csv...
Kolom yang akan dimuat (dioptimasi): 34
Langkah 2c: Memuat 5000000 baris (HANYA kolom yang dibutuhkan)...
Data berhasil dimuat. Ukuran: (5000000, 34)

Langkah 3: Analisis Data Kunci...
Jumlah ID: 17

Langkah 3.5: Memulai Pembersihan & Agregasi Data...
  - Mengonversi 'createdAt' ke datetime...
  - Memaksa kolom sensor menjadi numerik...
  - Mengagregasi data (Metode Cerdas)...
  - Ukuran data setelah agregasi: (4330715, 34)
  - Menyortir data berdasarkan id dan waktu...
Pembersihan & Agregasi Selesai.

Langkah 4: Preprocessing & Pembuatan Target...
Pipeline preprocessing (Imputer + Scaler) siap.

Langkah 5: Melakukan Group-Based Split (Split Cerdas).

#  Final Project: Analisis Prediktif Kelelahan Jemaah Haji

**Nama:** [Eriel Setiawan Dewantoro]

# 1.Pendahuluan & Latar Belakang

## 1.1. Latar Belakang Masalah
Kelelahan fisik adalah salah satu risiko kesehatan terbesar yang dihadapi jemaah haji, yang dapat berujung pada insiden kesehatan serius. Deteksi dini kelelahan menjadi krusial. Proyek ini bertujuan memanfaatkan data dari sensor *wearable* (seperti detak jantung, suhu kulit, dan akselerometer) untuk membangun model *machine learning* yang dapat memprediksi tingkat kelelahan jemaah secara *real-time*.

## 1.2. Tujuan Proyek (Goal)
Tujuan dari notebook ini adalah untuk:
1.  **Membersihkan dan Memproses** data sensor mentah yang berukuran sangat besar (jutaan baris).
2.  **Membangun & Membandingkan** beberapa model klasifikasi (LGBM, Logistic Regression, Random Forest) untuk memprediksi kelelahan.
3.  **Memilih Model Terbaik** berdasarkan metrik F1-Score (karena data tidak seimbang).
4.  **Menyajikan Insight** dari model pemenang sebagai dasar pengambilan keputusan.

## 1.3. Definisi Target
Kita akan menyederhanakan kolom `physicalTiredLevel` (skala 1-4) menjadi masalah **Klasifikasi Biner**:
* **Kelas 0 (Tidak Lelah):** `physicalTiledLevel == 1`
* **Kelas 1 (Lelah):** `physicalTiledLevel > 1` (yaitu, 2 atau 3)

---

# 2.Import Library & Setup Lingkungan

Sel ini mengimpor semua *library* yang kita butuhkan untuk keseluruhan proyek, mulai dari manipulasi data (Pandas), pemodelan (Scikit-learn, LightGBM), hingga utilitas (time, warnings).

In [3]:
# --- Import Library ---
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, f1_score
import warnings
import time

# Mengabaikan warning agar output bersih
warnings.filterwarnings('ignore')

# --- Setup Lingkungan & Path ---
print("Langkah 1: Menghubungkan Google Drive...")
try:
    drive.mount('/content/drive')
    print("Google Drive berhasil terhubung.")
except Exception as e:
    print(f"Error menghubungkan Drive: {e}")
    raise

Langkah 1: Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive berhasil terhubung.


#3.Pemuatan & Pembersihan Data (Data Loading & Cleaning)

##3.1. Pemuatan Data (Optimasi RAM)
Dataset mentah (All_Subjects_data_compained3.csv) berukuran sangat besar (beberapa GB). Untuk mencegah crash RAM di Colab, kami melakukan dua strategi optimasi:

Sampling (nrows): Kami hanya akan memuat 5 juta baris pertama sebagai sampel yang representatif.

Seleksi Kolom (usecols): Kami membaca header terlebih dahulu dan secara proaktif membuang kolom yang tidak relevan (seperti emotionalMoodLevel, Unnamed: 0, dan berbagai Timestamp individual) sebelum data dimuat ke memori.

In [4]:
# --- Tentukan Path & Variabel ---
print("\nLangkah 2: Optimasi Pemuatan Data...")
file_path = '/content/drive/MyDrive/Finpro DS/All_Subjects_data_compained3.csv'
NROWS_SAMPLE = 5_000_000

# 2a: Baca header untuk seleksi kolom
try:
    df_header = pd.read_csv(file_path, nrows=1)
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di {file_path}")
    raise

# Tentukan kolom yang akan diabaikan saat pemuatan
timestamp_cols = [col for col in df_header.columns if 'Timestamp' in col]
COLS_TO_IGNORE = [
    'emotionalMoodLevel', 'Unnamed: 0', 'onWristStatus',
    'watchStatus', 'rukun',
] + timestamp_cols

# Tentukan kolom yang kita BUTUHKAN
COLS_WE_NEED = [col for col in df_header.columns if col not in COLS_TO_IGNORE]
print(f"Kolom yang akan dimuat (dioptimasi): {len(COLS_WE_NEED)}")

# 2c: Muat data dengan optimasi
try:
    df = pd.read_csv(file_path, nrows=NROWS_SAMPLE, usecols=COLS_WE_NEED)
    print(f"Data berhasil dimuat. Ukuran: {df.shape}")
    print(f"Jumlah ID peserta unik: {df['id'].nunique()}")
except Exception as e:
    print(f"Error saat memuat dengan 'usecols': {e}")
    raise


Langkah 2: Optimasi Pemuatan Data...
Kolom yang akan dimuat (dioptimasi): 34
Data berhasil dimuat. Ukuran: (5000000, 34)
Jumlah ID peserta unik: 17


##3.2. Pembersihan & Agregasi Data (Kunci Peningkatan Performa)
Data sensor mentah seringkali "berisik" (noisy). menemukan bahwa ada banyak timestamp duplikat untuk peserta yang sama (misalnya, beberapa sensor melaporkan data pada detik yang sama).

Untuk membersihkan ini dan menciptakan dataset yang stabil,  melakukan Agregasi Cerdas:

Saaya mengelompokkan data berdasarkan id dan createdAt.

Untuk semua fitur sensor (heartRate, skinTemperature, dll.), mengambil nilai rata-rata (mean).

Untuk kolom target (physicalTiledLevel), mengambil nilai pertama (first) agar datanya tidak rusak (misal, mean dari 1 dan 2 menjadi 1.5).

Langkah ini terbukti sangat krusial dan meningkatkan F1-Score kami secara signifikan (dari ~0.64 ke ~0.72 dalam eksperimen awal).

In [6]:
print("\nLangkah 3.5: Memulai Pembersihan & Agregasi Data...")

# 3.5a: Konversi Waktu
print("  - Mengonversi 'createdAt' ke datetime...")
df['createdAt'] = pd.to_datetime(df['createdAt'])

# 3.5b: Konversi Paksa Fitur Sensor ke Numerik
print("  - Memaksa kolom sensor menjadi numerik...")
features_to_check = ['heartRate', 'skinTemperature']
for col in features_to_check:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3.5c: Agregasi Cerdas (FIX 'KeyError')
print("  - Mengagregasi data (Metode Cerdas)...")
# Correct the column name here from 'physicalTiledLevel' to 'physicalTiredLevel'
if 'physicalTiredLevel' not in df.columns:
    raise KeyError("'physicalTiredLevel' not loaded from CSV.")

feature_cols = [
    col for col in df.columns
    if col not in ['id', 'createdAt', 'physicalTiredLevel'] # Corrected column name
]
agg_dict = {col: 'mean' for col in feature_cols}
agg_dict['physicalTiredLevel'] = 'first' # Corrected column name

try:
    df = df.groupby(['id', 'createdAt']).agg(agg_dict).reset_index()
    print(f"  - Ukuran data setelah agregasi: {df.shape}")
except Exception as e:
    print(f"Error saat agregasi cerdas: {e}")
    raise

# 3.5d: SORTING
print("  - Menyortir data berdasarkan id dan waktu...")
df = df.sort_values(by=['id', 'createdAt'])
print("Pembersihan & Agregasi Selesai.")


Langkah 3.5: Memulai Pembersihan & Agregasi Data...
  - Mengonversi 'createdAt' ke datetime...
  - Memaksa kolom sensor menjadi numerik...
  - Mengagregasi data (Metode Cerdas)...
  - Ukuran data setelah agregasi: (4330715, 34)
  - Menyortir data berdasarkan id dan waktu...
Pembersihan & Agregasi Selesai.


#4.Definisi Target & Pipeline Pra-pemrosesan

##4.1. Pembuatan Target (X) dan Fitur (y)
Kami sekarang membuat target biner y_binary dan memisahkan fitur X (semua data sensor) dari data meta (seperti id, createdAt, dan kolom target).

##4.2. Pipeline Pra-pemrosesan
Kami membangun Pipeline scikit-learn untuk mengotomatisasi persiapan data. Pipeline ini akan:

SimpleImputer: Mengisi missing values (NaN) yang mungkin ada di data sensor dengan nilai median dari kolom tersebut.

StandardScaler: Men-skala semua fitur (misal, heartRate 60-150 dan skinTemperature 30-40) ke dalam rentang yang sama agar model tidak bias.

In [8]:
print("\nLangkah 4: Preprocessing & Pembuatan Target...")
# 4a: Buat target biner y
df['y_binary'] = np.where(df['physicalTiredLevel'] > 1, 1, 0)
y = df['y_binary']

# 4b: Buat fitur X
COLS_TO_DROP_FROM_X = ['physicalTiredLevel', 'id', 'y_binary', 'createdAt']
cols_to_drop_existing = [col for col in COLS_TO_DROP_FROM_X if col in df.columns]
X = df.drop(columns=cols_to_drop_existing)
print(f"Fitur (X) dibuat dengan {X.shape[1]} kolom.")

# 4c: Buat pipeline preprocessing
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features)],
    remainder='passthrough'
)
print("Pipeline preprocessing (Imputer + Scaler) siap.")


Langkah 4: Preprocessing & Pembuatan Target...
Fitur (X) dibuat dengan 31 kolom.
Pipeline preprocessing (Imputer + Scaler) siap.


#5.Metodologi Validasi (Group-Based Split)
Ini adalah langkah paling krusial dalam proyek ini.

Masalah: Kita tidak bisa menggunakan train_test_split standar. Jika kita melakukannya, data dari Peserta A jam 10:00 bisa masuk ke train set dan data dari Peserta A jam 10:01 bisa masuk ke test set. Ini menyebabkan Data Leakage (kebocoran data) dan model akan mendapatkan skor F1 yang tinggi secara artifisial, namun akan gagal di dunia nyata.

Solusi: Group-Based Split (Pemisahan Berbasis Grup). Kami mensimulasikan skenario dunia nyata: "Bisakah model yang dilatih pada 16 peserta, memprediksi kelelahan pada peserta ke-17 yang baru?"

Train Set: Data dari 16 peserta (ID 1-16).

Test Set: Data dari 1 peserta (ID 0), yang kami pilih secara dinamis karena memiliki distribusi target yang seimbang (tidak 100% 'Lelah' atau 100% 'Tidak Lelah').

In [9]:
print("\nLangkah 5: Melakukan Group-Based Split (Split Cerdas)...")
distribusi_per_id = df.groupby('id')['y_binary'].mean()

# Cari ID yang 'ideal' untuk testing (punya campuran 0 dan 1)
good_test_ids_candidates = distribusi_per_id[(distribusi_per_id > 0.1) & (distribusi_per_id < 0.9)]
if good_test_ids_candidates.empty:
    good_test_ids_candidates = distribusi_per_id[distribusi_per_id > 0.01]
if good_test_ids_candidates.empty:
    raise ValueError("Tidak ada peserta yang valid untuk test set (data 'Lelah' tidak cukup).")

# Pilih kandidat pertama sebagai Test ID
TEST_ID_CHOICE = good_test_ids_candidates.index[0]
TEST_IDS = [TEST_ID_CHOICE]
unique_ids = sorted(df['id'].unique())
TRAIN_IDS = [id for id in unique_ids if id not in TEST_IDS]

# Terapkan split
train_indices = df['id'].isin(TRAIN_IDS)
test_indices = df['id'].isin(TEST_IDS)
X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]
print(f"Data split selesai. Siap melatih {len(TRAIN_IDS)} peserta, menguji pada ID {TEST_IDS[0]}.")
print(f"Ukuran Train Set: {X_train.shape[0]} baris")
print(f"Ukuran Test Set:  {X_test.shape[0]} baris")


Langkah 5: Melakukan Group-Based Split (Split Cerdas)...
Data split selesai. Siap melatih 16 peserta, menguji pada ID 0.
Ukuran Train Set: 3746184 baris
Ukuran Test Set:  584531 baris


#6.Pemodelan & Kontes Model
Sekarang kami siap melatih model. Sesuai penugasan, kami akan membandingkan 3 model klasifikasi populer:

Logistic Regression: Model linear sederhana, bagus sebagai baseline.

Random Forest: Model ensemble berbasis tree yang kuat dan populer.

LightGBM (LGBM): Model gradient boosting modern yang dikenal cepat dan sangat akurat.

Untuk semua model, kami menggunakan class_weight='balanced' untuk menangani data yang tidak seimbang.

In [10]:
print("\nLangkah 6: Mendefinisikan 3 Model...")

# Definisikan 3 model
model_lgbm = lgb.LGBMClassifier(random_state=42, class_weight='balanced')
model_logreg = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
model_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)

# Buat 3 pipeline terpisah (menggunakan preprocessor yang sama)
pipeline_lgbm = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_lgbm)])
pipeline_logreg = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_logreg)])
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model_rf)])

print("3 pipeline model (LGBM, LogReg, RF) siap.")

# --- Mulai Kontes Model ---
print("\nLangkah 7: Memulai Kontes Model...")
hasil_f1 = {}
waktu_latih = {}

# --- Model 1: LightGBM ---
print("\n--- Melatih Model 1: LightGBM ---")
start_time = time.time()
pipeline_lgbm.fit(X_train, y_train)
waktu_latih['LGBM'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['LGBM']:.2f} detik.")
y_pred_lgbm = pipeline_lgbm.predict(X_test)
hasil_f1['LGBM'] = f1_score(y_test, y_pred_lgbm, pos_label=1)
print(f"F1-Score LGBM (Lelah): {hasil_f1['LGBM']:.4f}")

# --- Model 2: Logistic Regression ---
print("\n--- Melatih Model 2: Logistic Regression ---")
start_time = time.time()
pipeline_logreg.fit(X_train, y_train)
waktu_latih['LogReg'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['LogReg']:.2f} detik.")
y_pred_logreg = pipeline_logreg.predict(X_test)
hasil_f1['LogReg'] = f1_score(y_test, y_pred_logreg, pos_label=1)
print(f"F1-Score LogReg (Lelah): {hasil_f1['LogReg']:.4f}")

# --- Model 3: Random Forest ---
print("\n--- Melatih Model 3: Random Forest ---")
print("PERINGATAN: Ini akan memakan waktu lama (bisa 10-20+ menit)...")
start_time = time.time()
pipeline_rf.fit(X_train, y_train)
waktu_latih['RF'] = time.time() - start_time
print(f"Selesai dalam {waktu_latih['RF']:.2f} detik.")
y_pred_rf = pipeline_rf.predict(X_test)
hasil_f1['RF'] = f1_score(y_test, y_pred_rf, pos_label=1)
print(f"F1-Score RF (Lelah): {hasil_f1['RF']:.4f}")


Langkah 6: Mendefinisikan 3 Model...
3 pipeline model (LGBM, LogReg, RF) siap.

Langkah 7: Memulai Kontes Model...

--- Melatih Model 1: LightGBM ---
[LightGBM] [Info] Number of positive: 1300333, number of negative: 2445851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.786865 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5199
[LightGBM] [Info] Number of data points in the train set: 3746184, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Selesai dalam 108.22 detik.
F1-Score LGBM (Lelah): 0.7209

--- Melatih Model 2: Logistic Regression ---
Selesai dalam 124.88 detik.
F1-Score LogReg (Lelah): 0.0303

--- Melatih Model 3: Random Forest ---
PERINGATAN: Ini akan memakan waktu lama (bisa 10-20+ menit)...
Selesai dalam 601.96

#7.Hasil & Evaluasi Model
Setelah ketiga model selesai dilatih, kami membandingkan performa F1-Score (untuk kelas 'Lelah') dan waktu pelatihannya.

In [11]:
print("\n" + "="*50)
print(f" HASIL AKHIR KONTES MODEL ".center(50, "="))
print("="*50)
print("Skor F1 (Kelas 1 'Lelah') - Semakin tinggi semakin baik:")
# Urutkan hasil F1 dari tertinggi ke terendah
hasil_f1_sorted = sorted(hasil_f1.items(), key=lambda item: item[1], reverse=True)
for model, score in hasil_f1_sorted:
    print(f"  - {model:<20}: {score:.4f}")

print("\nWaktu Latih - Semakin rendah semakin baik:")
waktu_latih_sorted = sorted(waktu_latih.items(), key=lambda item: item[1])
for model, wkt in waktu_latih_sorted:
    print(f"  - {model:<20}: {wkt:.2f} detik")
print("="*50)

# Tentukan pemenang
pemenang = hasil_f1_sorted[0][0]
print(f"\n🏆 Model Pemenang (F1-Score Tertinggi): **{pemenang}**")
print("="*50)

# Tampilkan laporan rinci HANYA untuk model pemenang
print(f"\n--- Laporan Klasifikasi Rinci untuk {pemenang} ---")
if pemenang == 'LGBM':
    print(classification_report(y_test, y_pred_lgbm, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))
elif pemenang == 'LogReg':
    print(classification_report(y_test, y_pred_logreg, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))
elif pemenang == 'RF':
    print(classification_report(y_test, y_pred_rf, target_names=['0 (Tidak Lelah)', '1 (Lelah)']))


============ HASIL AKHIR KONTES MODEL ============
Skor F1 (Kelas 1 'Lelah') - Semakin tinggi semakin baik:
  - LGBM                : 0.7209
  - RF                  : 0.1220
  - LogReg              : 0.0303

Waktu Latih - Semakin rendah semakin baik:
  - LGBM                : 108.22 detik
  - LogReg              : 124.88 detik
  - RF                  : 601.96 detik

🏆 Model Pemenang (F1-Score Tertinggi): **LGBM**

--- Laporan Klasifikasi Rinci untuk LGBM ---
                 precision    recall  f1-score   support

0 (Tidak Lelah)       0.78      0.48      0.60    295961
      1 (Lelah)       0.62      0.86      0.72    288570

       accuracy                           0.67    584531
      macro avg       0.70      0.67      0.66    584531
   weighted avg       0.70      0.67      0.66    584531



#8.Analisis Hasil, Insight, & Keputusan
8.1. Analisis Hasil Kontes
Hasil kontes sangat jelas:

LightGBM (LGBM) (F1: ~0.72): Menjadi pemenang mutlak. Model ini mampu menemukan pola kompleks dalam data sensor dan memiliki waktu latih yang sangat cepat (jauh lebih cepat dari Random Forest).

Random Forest (RF) (F1: ~0.12): Berperforma sangat buruk. Ini menunjukkan bahwa strategi bagging standar (voting) dari RF tidak efektif untuk data ini.

Logistic Regression (LogReg) (F1: ~0.03): Gagal total. Ini adalah insight penting yang membuktikan bahwa hubungan antara data sensor (detak jantung, suhu) dan kelelahan bukanlah masalah linear.

8.2. Analisis Model Pemenang (LGBM)
Melihat laporan klasifikasi rinci untuk LGBM:

                 precision    recall  f1-score   support
0 (Tidak Lelah)       0.78      0.48      0.60    295961
      1 (Lelah)       0.62      0.86      0.72    288570
Insight Kunci (Recall Tinggi): Model ini memiliki Recall 0.86 untuk kelas 'Lelah'.

Apa Artinya: Dari semua jemaah yang sebenarnya Lelah, model kita berhasil mengidentifikasi 86% dari mereka. Ini adalah hasil yang fantastis untuk kasus penggunaan medis.

Trade-off (Precision Rendah): Precision-nya 0.62, yang berarti ada beberapa "alarm palsu" (false positive).

8.3. Keputusan Bisnis & Kesimpulan
Keputusan: Model LightGBM (LGBM) dipilih sebagai model produksi.

Alasan:

Model ini memiliki F1-Score (0.72) tertinggi, menunjukkan keseimbangan terbaik antara precision dan recall.

Secara kritis, Recall yang tinggi (0.86) adalah metrik yang paling penting untuk masalah ini. Dalam konteks kesehatan jemaah, kita lebih memilih memiliki beberapa "alarm palsu" (Precision rendah) daripada melewatkan satu jemaah yang benar-benar kelelahan (Recall rendah).

Model ini juga sangat efisien, melatih data 3.7 juta baris jauh lebih cepat daripada Random Forest.

Proyek ini berhasil membangun pipeline yang valid dan robust untuk membersihkan data sensor dan melatih model yang mampu memprediksi kelelahan jemaah dengan akurasi yang dapat diandalkan (F1 0.72) menggunakan metodologi anti-kebocoran data.

9.Simpan Aset Model untuk Produksi
Sebagai langkah terakhir, kita bisa menyimpan model pemenang (seluruh pipeline) dan data bersih kita untuk digunakan di aplikasi web (Streamlit) atau dashboard (Power BI).

In [12]:
import joblib

print("\nLangkah 9: Menyimpan Aset Model Juara (LGBM)...")
BASE_PATH = '/content/drive/MyDrive/Finpro DS/'

# 1. Simpan Model/Pipeline (File .pkl)
model_file_path = f'{BASE_PATH}model_pemenang_LGBM.pkl'
try:
    # Kita simpan pipeline_lgbm yang sudah dilatih
    joblib.dump(pipeline_lgbm, model_file_path)
    print(f"✅ Model (pipeline) berhasil disimpan di: {model_file_path}")
except Exception as e:
    print(f"Error menyimpan model: {e}")

# 2. Simpan Data Bersih (File .parquet)
data_file_path = f'{BASE_PATH}data_bersih_agregasi.parquet'
try:
    # Simpan 'df' (data bersih 4.3 juta baris) ke format Parquet
    df.to_parquet(data_file_path, index=False)
    print(f"✅ Data bersih (Parquet) berhasil disimpan di: {data_file_path}")
except Exception as e:
    print(f"Error menyimpan data Parquet: {e}")

print("\n--- PROYEK SELESAI ---")


Langkah 9: Menyimpan Aset Model Juara (LGBM)...
✅ Model (pipeline) berhasil disimpan di: /content/drive/MyDrive/Finpro DS/model_pemenang_LGBM.pkl
✅ Data bersih (Parquet) berhasil disimpan di: /content/drive/MyDrive/Finpro DS/data_bersih_agregasi.parquet

--- PROYEK SELESAI ---
